In [31]:
import polars as pl
import os
from quant101.core_2.config import data_dir
'''
this script is to fix the day agg data issue where NVDA's high value is wrong
by replacing it with the max high value from minute agg data
'''

minute_data_dir = os.path.join(data_dir, "lake/us_stocks_sip/minute_aggs_v1/2024/06/2024-06-10.parquet")
day_data_dir = os.path.join(data_dir, "lake/us_stocks_sip/day_aggs_v1/2024/06/2024-06-10.parquet")

df_minute = pl.read_parquet(minute_data_dir)

actual_max_high = df_minute.group_by('ticker').agg(pl.col('high').max().alias('max_high')).filter(
    (pl.col("ticker") == "NVDA")
).drop('ticker').to_series().to_list()[0]
print(actual_max_high)  # 123.099998

# with pl.Config(tbl_cols=200, tbl_rows=200, fmt_str_lengths=200, tbl_width_chars=3000, tbl_formatting="UTF8_FULL_CONDENSED"):
#     print(df.head(20))

df_day = pl.read_parquet(day_data_dir)
df_fixed = df_day.with_columns(
    pl.when(pl.col('ticker') == 'NVDA').then(pl.lit(actual_max_high)).otherwise(pl.col('high')).alias('high')
)

after_max_high = df_fixed.group_by('ticker').agg(pl.col('high').max().alias('max_high')).filter(
    (pl.col("ticker") == "NVDA")
).drop('ticker').to_series().to_list()[0]
print(after_max_high)


tsla_after_max_high = df_fixed.group_by('ticker').agg(pl.col('high').max().alias('max_high')).filter(
    (pl.col("ticker") == "TSLA")
).drop('ticker').to_series().to_list()[0]

print(tsla_after_max_high)

# df_fixed.write_parquet(day_data_dir)

123.0999984741211
123.0999984741211
178.57000732421875


In [ ]:
import polars as pl
from quant101.core_2.data_loader import data_dir_calculate
import os

lake_file_paths = data_dir_calculate(asset='us_stock_sip', data_type='minute_aggs_v1', start_date='2025-07-01', end_date='2025-07-31', lake=True)

lf = pl.scan_parquet(
        lake_file_paths,
    )

# 计算 VWAP (每个 ticker)
vwap = (
    lf.filter(pl.col("volume") > 0)
        .group_by("ticker")
        .agg((pl.col("close") * pl.col("volume")).sum() / pl.col("volume").sum())
        .rename({"close": "vwap"})
)

# print(vwap.collect().sort('ticker').head())


# 找到每个 ticker 当月最后的 close
last_close = (
    lf.sort("window_start")
      .group_by("ticker")
      .tail(1)
      .select(["ticker", "close"])
      .rename({"close": "last_close"})
)

# print(last_close.collect().sort('ticker').head())

# 合并结果，计算比率
result = (
    vwap.join(last_close, on="ticker")
        .with_columns(
            (
                pl.col("last_close") / pl.col("vwap")).alias("ratio"),
                (pl.col("last_close") - pl.col("vwap")).abs().alias("vwap_diff"),
                ((pl.col("last_close") - pl.col("vwap")) / pl.col("vwap")).abs().alias("ratio_stability_1"),
                abs((pl.col("last_close") - pl.col("vwap")) / pl.col("vwap")).cast(pl.Float32).alias("ratio_stability_2")
             )
        .sort("ratio_stability_1",'last_close', descending=[False, True])
)

print(result.collect().head(5))

shape: (5, 7)
┌────────┬────────────┬────────────┬───────┬───────────┬───────────────────┬───────────────────┐
│ ticker ┆ vwap       ┆ last_close ┆ ratio ┆ vwap_diff ┆ ratio_stability_1 ┆ ratio_stability_2 │
│ ---    ┆ ---        ┆ ---        ┆ ---   ┆ ---       ┆ ---               ┆ ---               │
│ str    ┆ f64        ┆ f32        ┆ f64   ┆ f64       ┆ f64               ┆ f32               │
╞════════╪════════════╪════════════╪═══════╪═══════════╪═══════════════════╪═══════════════════╡
│ ESGR   ┆ 337.910004 ┆ 337.910004 ┆ 1.0   ┆ 0.0       ┆ 0.0               ┆ 0.0               │
│ FIG    ┆ 115.5      ┆ 115.5      ┆ 1.0   ┆ 0.0       ┆ 0.0               ┆ 0.0               │
│ ZXIET  ┆ 100.0      ┆ 100.0      ┆ 1.0   ┆ 0.0       ┆ 0.0               ┆ 0.0               │
│ LBDAV  ┆ 91.860001  ┆ 91.860001  ┆ 1.0   ┆ 0.0       ┆ 0.0               ┆ 0.0               │
│ AEBIV  ┆ 83.260002  ┆ 83.260002  ┆ 1.0   ┆ 0.0       ┆ 0.0               ┆ 0.0               │
└────────┴──────

In [1]:
import duckdb

data_dir = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/07/*.parquet"
con = duckdb.connect()

query = f"""
WITH vwap AS (
    SELECT 
        ticker,
        SUM(close * volume)::DOUBLE / NULLIF(SUM(volume),0) AS vwap
    FROM '{data_dir}'
    GROUP BY ticker
),
last_close AS (
    SELECT DISTINCT ON (ticker)
        ticker, close AS last_close
    FROM '{data_dir}'
    ORDER BY ticker, window_start DESC
)
SELECT 
    vwap.ticker,
    vwap.vwap,
    last_close.last_close,
    last_close.last_close / vwap.vwap AS ratio
FROM vwap
JOIN last_close USING (ticker)
ORDER BY ratio DESC
LIMIT 1;
"""

result = con.sql(query).df()
print(result)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  ticker      vwap  last_close      ratio
0    GVH  0.133029        5.56  41.795326


In [1]:
import polars as pl
import exchange_calendars as xcals

# 纽约证券交易所 (美国)
xnys = xcals.get_calendar('XNYS')
snys_schedule = xnys.schedule.loc['2005-01-01':'2025-12-31'] 

# 转换为 Polars DataFrame
df_schedule = pl.from_pandas(snys_schedule.reset_index())

df_schedule = df_schedule.with_columns([
    pl.col('open').dt.convert_time_zone('America/New_York'),
    pl.col('close').dt.convert_time_zone('America/New_York')
])

with pl.Config(tbl_cols=200, fmt_str_lengths=200, tbl_width_chars=3000, tbl_formatting="UTF8_FULL_CONDENSED"):
    df_schedule_filtered = df_schedule.filter(
        pl.col('close').dt.hour() != 16
    )
    print(df_schedule_filtered)

shape: (45, 5)
┌─────────────────────┬────────────────────────────────┬───────────────────┬───────────────────┬────────────────────────────────┐
│ index               ┆ open                           ┆ break_start       ┆ break_end         ┆ close                          │
│ ---                 ┆ ---                            ┆ ---               ┆ ---               ┆ ---                            │
│ datetime[ns]        ┆ datetime[ns, America/New_York] ┆ datetime[ns, UTC] ┆ datetime[ns, UTC] ┆ datetime[ns, America/New_York] │
╞═════════════════════╪════════════════════════════════╪═══════════════════╪═══════════════════╪════════════════════════════════╡
│ 2005-11-25 00:00:00 ┆ 2005-11-25 09:30:00 EST        ┆ null              ┆ null              ┆ 2005-11-25 13:00:00 EST        │
│ 2006-07-03 00:00:00 ┆ 2006-07-03 09:30:00 EDT        ┆ null              ┆ null              ┆ 2006-07-03 13:00:00 EDT        │
│ 2006-11-24 00:00:00 ┆ 2006-11-24 09:30:00 EST        ┆ null              

In [3]:
import polars as pl
from quant101.core_2.data_loader import data_dir_calculate
import os

# Get the file paths
lake_file_paths = data_dir_calculate(
    asset='us_stocks_sip', 
    data_type='day_aggs_v1', 
    start_date='2024-06-10',
    end_date='2024-06-10',
    lake=True
)

if not lake_file_paths:
        raise ValueError(
            f"No data files found."
        )

tickers = 'NVDA'
tickers = [t.strip().upper() for t in tickers.split(",")] if tickers else None
if tickers:
    lf = pl.scan_parquet(lake_file_paths).filter(pl.col('ticker').is_in(tickers))
else:
    lf = pl.scan_parquet(lake_file_paths)

lf = lf.with_columns(
    pl.from_epoch(pl.col('window_start'), time_unit='ns')
    .dt.convert_time_zone('America/New_York')
    .alias('timestamps')

).sort('timestamps').collect()
with pl.Config(tbl_cols=10):
    print(lf.head())


shape: (1, 9)
┌────────┬───────────┬───────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ ticker ┆ volume    ┆ open      ┆ close    ┆ high     ┆ low      ┆ window_s ┆ transact ┆ timestam │
│ ---    ┆ ---       ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ tart     ┆ ions     ┆ ps       │
│ str    ┆ u64       ┆ f32       ┆ f32      ┆ f32      ┆ f32      ┆ ---      ┆ ---      ┆ ---      │
│        ┆           ┆           ┆          ┆          ┆          ┆ i64      ┆ u32      ┆ datetime │
│        ┆           ┆           ┆          ┆          ┆          ┆          ┆          ┆ [ns, Ame │
│        ┆           ┆           ┆          ┆          ┆          ┆          ┆          ┆ rica/New │
│        ┆           ┆           ┆          ┆          ┆          ┆          ┆          ┆ _York]   │
╞════════╪═══════════╪═══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ NVDA   ┆ 314162650 ┆ 120.37000 ┆ 121.7900 ┆ 195.9499 ┆ 117.0100 ┆ 17179920 

In [ ]:
from quant101.core_2.data_loader import stock_load_process
import polars as pl
from quant101.core_2.plotter import plot_candlestick

tickers = ['NVDA']
# tickers = None
timeframe = "1h" # timeframe: '1m', '3m', '5m', '10m', '15m', '20m', '30m', '45m', '1h', '2h', '3h', '4h', '1d' 等
asset = 'us_stocks_sip'
data_type = ('day_aggs_v1' if timeframe == '1d' else 'minute_aggs_v1')
start_date = "2025-07-01"
end_date = "2025-07-08"
full_hour = True

lf_result = stock_load_process(
        tickers=tickers, 
        timeframe=timeframe, 
        asset=asset, 
        data_type=data_type, 
        start_date=start_date, 
        end_date=end_date,
        full_hour=full_hour
    ).collect()

print(lf_result.head())

# plot_candlestick(lf_result.to_pandas(), ticker=tickers, timeframe=timeframe)